In [66]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings("ignore")
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, matthews_corrcoef, median_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict
from sklearn import cross_validation
import matplotlib.pyplot as plt
from sklearn import model_selection
import import_ipynb
from save_lib import save_RF_model
from save_lib import save_NN_model
import time
from sklearn.neural_network import MLPRegressor
import os
from sklearn.preprocessing import LabelEncoder
import theano
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [84]:
ls_need_col = [
#     'Поставщик',
#     'НД (гост, ту)',
#     'ГОСТ, ТУ',
#     'Дата форм. цехом',
#     'марка стали',
#     '№ плавки',
#     'кол-во, шт.',
#     '№ партии',
    '1 зона по ВТР закалка', 
#     '1 зона уставка закалка',
#     '1 зона фактическая температура закалка', 
#     '2 зона по ВТР закалка',
    '3 зона по ВТР закалка', 
#     '3 зона уставка закалка',
#     '3 зона фактическая температура закалка',
#     'Температура трубы на выгрузке из печи закалки',
    'Скорость прохождения трубы через спрейер, м/с',
    't˚ C трубы после спреера', 
#     'Расход воды на закалочный спрейер, м3',
#     'время выдержки в закалочной ванне, сек.',
#     't˚ C воды в закалочной ванне', 
#     't˚ C трубы после ванны',
    '1 зона ВТР и уставка отпуск', 
#     '1 зона фактическая температура отпуск',
    '2 зона ВТР и уставка отпуск', 
#     '3 зона ВТР и уставка отпуск',
#     '3 зона фактическая температура отпуск', 
#     '4 зона ВТР и уставка отпуск',
#     '5 зона ВТР и уставка отпуск', 
#     '5 зона фактическая температура отпуск',
#     'Температура трубы на выгрузке из печи отпуска',
    'шаг балок закалочная печь, сек', 
    'шаг балок отпускная печь, сек',
    'диаметр', 
    'толщина стенки', 
    'C', 
    'Mn', 
    'Si', 
    'P', 
    'S',
    'Cr', 
    'Ni', 
    'Cu', 
    'Al', 
    'V', 
#     'Ti', 
#     'Nb', 
#     'Mo', 
#     'N', 
#     'B',
    'Параметр отпуск',
    'Параметр закалка',
#     'Тип предела текучести 0.2',
#     'Тип предела текучести 0.5',
#     'Тип предела текучести 0.6',
    'C-coef',
    'ICD'
]

df_prepared_merge = pd.DataFrame()

for col in ls_need_col+['Предел текучести', 'Врем. сопротивление']:
    df_prepared_merge[col] = df_merge[col]
    
df_prepared_valid = pd.DataFrame()

for col in ls_need_col+['Предел текучести', 'Врем. сопротивление']: #ls_steel
    df_prepared_valid[col] = df_merge_valid[col]

# ls_need_col.remove('марка стали')

In [62]:
df_merge = pd.read_excel('prepared_hard.xlsx')
df_merge_valid = pd.read_excel('prepared_hard_valid.xlsx')

In [67]:
df_merge.shape, df_merge_valid.shape

((21518, 54), (452, 43))

In [68]:
ls_steel = pd.get_dummies(df_merge['марка стали']).columns
ls_steel = list(ls_steel)

In [85]:
'''
    70.5(690.9)    30ХГМА
    77.75(762)     37Г2Ф
    69.64(682.5)   30Г2
''' 
d = {
#     'НД (гост, ту)' : [ ]
    'марка стали' : [35, 52, 27], #'30ХГМА', '37Г2Ф', '30Г2'
    '1 зона по ВТР закалка' : [860, 840, 850], 
#     '2 зона по ВТР закалка' : [860, 840, 850],
    '3 зона по ВТР закалка' : [860, 850, 860], 
#     'Температура трубы на выгрузке из печи закалки' : [855, 835, 850],
    'Скорость прохождения трубы через спрейер, м/с' : [0.15, 0.28, 0.37],
    't˚ C трубы после спреера' : [106, 100, 75],
    '1 зона ВТР и уставка отпуск' : [675, 664, 600], 
    '2 зона ВТР и уставка отпуск' : [675, 664, 600], 
    '3 зона ВТР и уставка отпуск' : [650, 653, 593],
    '4 зона ВТР и уставка отпуск' : [650, 653, 593],
    '5 зона ВТР и уставка отпуск' : [633, 637, 580], 
#     'Температура трубы на выгрузке из печи отпуска' : [660, 660, 575],
    'шаг балок закалочная печь, сек' : [37, 40, 30], 
    'шаг балок отпускная печь, сек' : [37, 31, 30],
    'диаметр' : [187.71, 88.9, 168.28], 
    'толщина стенки' : [16.5, 12.2, 8.94], 
    'C' : [0.29, 0.37, 0.28], 
    'Mn' : [0.92, 1.49, 1.33], 
    'Si' : [0.26, 0.28, 0.26], 
    'P' : [0.009, 0.011, 0.009], 
    'S' : [0.005, 0.007, 0.007],
    'Cr' : [0.68, 0.14, 0.1], 
    'Ni' : [0.13, 0.13, 0.13], 
    'Cu' : [0.15, 0.13, 0.19], 
    'Al' : [0.03, 0.03, 0.03], 
    'V' : [0.007, 0.09, 0.005], 
#     'Ti' : [0, 0.006, 0.007], 
#     'Nb' : [0, 0.0002, 0.006], 
#     'Mo' : [0.16, 0.0009, 0.015], 
#     'N' : [0.008, 0.006, 0.009], 
#     'B': [0, 0, 0.0005],
    'Параметр отпуск' : [25.826798, 25.757751, 24.078837],
    'Параметр закалка' : [849.279448, 836.289772, 839.665069],
#     'Тип предела текучести 0.2': [0, 0, 0],
#     'Тип предела текучести 0.5': [0, 0, 0],
#     'Тип предела текучести 0.6': [0, 0, 0],
    'C-coef': [0.39, 0.37, 0.31],
    'ICD':[4.133959, 1.695309, 1.452618]
}

df_valid = pd.DataFrame(data = d)

In [86]:
df_valid = df_valid[ls_need_col]
df_valid.shape

(3, 24)

In [73]:
df_prepared_merge.shape

(21518, 27)

In [74]:
arr_df_steel = []
arr_valid_steel = []
for steel in ls_steel:
    arr_df_steel.append(df_prepared_merge[df_prepared_merge['марка стали'] == steel])
    arr_valid_steel.append(df_prepared_valid[df_prepared_valid['марка стали'] == steel])

len(arr_df_steel)

51

In [75]:
for df in arr_df_steel:
    print(df.shape, df['марка стали'].values[0])

(2, 27) 08ХГМФБ
(2, 27) 08ХГФБ
(214, 27) 09Г2С
(2, 27) 09ГФБ
(1, 27) 10Г2
(3, 27) 12ГА
(1305, 27) 13ХФА
(12, 27) 13ХФА-3
(4, 27) 13ХФА-4
(30, 27) 18Х3МФБ
(1878, 27) 18ХМФБ
(70, 27) 20
(22, 27) 20А
(28, 27) 20А-1
(11, 27) 20А-У
(2, 27) 20А-пр
(11, 27) 20Г2
(2, 27) 20Г2-1
(23, 27) 20ФА
(10, 27) 20ХФА
(5, 27) 26Г1ФМ
(11986, 27) 30Г2
(1211, 27) 30Г2-2
(378, 27) 30Г2-3
(52, 27) 30Г2-5
(617, 27) 30Г2-8
(4, 27) 30Г2Ф
(1, 27) 30Г3
(224, 27) 30ХГМА
(27, 27) 30ХГМА-1
(78, 27) 30ХГМА-4
(7, 27) 30ХГСА
(5, 27) 30ХМА
(9, 27) 32Г1
(13, 27) 32Г2
(2, 27) 32Г2СФБ
(8, 27) 32Х1МФ
(9, 27) 32Х1МФ-1
(3, 27) 32ХА
(5, 27) 32ХНМ
(2, 27) 36ХГ2СФБ
(1, 27) 36ХГФ
(2606, 27) 37Г2Ф
(1, 27) 37Г2Ф-М
(1, 27) 38Г2-4
(508, 27) 38Г2С
(5, 27) 38Г2С-3
(71, 27) 38Г2С-4
(40, 27) 38Г2С-9
(1, 27) 38Г2СФ
(6, 27) Д


In [76]:
len_arr = []
for x in range(0,len(arr_df_steel)):
    len_arr.append(x)

In [77]:
len_arr.reverse()

In [78]:
for i in len_arr:
    arr_df_steel[i] = arr_df_steel[i].dropna()
    arr_df_steel[i] =  arr_df_steel[i].reset_index(drop=True)
    arr_valid_steel[i] = arr_valid_steel[i].dropna()
    arr_valid_steel[i] =  arr_valid_steel[i].reset_index(drop=True)

for i in len_arr:
    if (arr_valid_steel[i].shape[0] < 2) | (arr_df_steel[i].shape[0] < 50):
        del arr_df_steel[i]
        del arr_valid_steel[i]

In [79]:
len(arr_df_steel)

5

In [80]:
for i in arr_df_steel:
    print(i.shape,i['марка стали'].values[0])

ls_steel_zip = []
print('\nValid:')
for i in arr_valid_steel:
    print(i.shape,i['марка стали'].values[0])
    ls_steel_zip.append(i['марка стали'].values[0])


(160, 27) 09Г2С
(755, 27) 13ХФА
(481, 27) 18ХМФБ
(3608, 27) 30Г2
(175, 27) 30Г2-8

Valid:
(20, 27) 09Г2С
(8, 27) 13ХФА
(6, 27) 18ХМФБ
(99, 27) 30Г2
(12, 27) 30Г2-8


In [195]:
from sklearn.learning_curve import learning_curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


# cv = cross_validation.ShuffleSplit(X.shape[0], n_iter=500,
#                                    test_size=0.2, random_state=0)
# title = 'title'
# plot_learning_curve(rfc, title, X, y, cv=cv, ylim=(0.7, 1.01), n_jobs=-1)

In [210]:
label = LabelEncoder() 
scaler = StandardScaler()  

for df, valid in zip(arr_df_steel, arr_valid_steel):
    print(df['марка стали'].values[0])

    y = pd.DataFrame()
    X = pd.DataFrame()
    y_valid = pd.DataFrame()
    X_valid = pd.DataFrame()
    
#     target = 'Предел текучести'
    target = 'Врем. сопротивление'

    y[target] = df[target]
    y_valid[target] = valid[target]

    for i in ls_need_col: #+ls_steel:
        X[i] = df[i]

    for i in ls_need_col: #+ls_steel:
        X_valid[i] = valid[i]
    
    X_valid = X_valid.dropna()

    
    X['марка стали'] = label.fit_transform(X['марка стали'])
    X_valid['марка стали'] = label.fit_transform(X_valid['марка стали'])
    X = scaler.fit_transform(X, y)
    X_valid = scaler.fit_transform(X_valid, y_valid)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    rfc = RandomForestRegressor(max_features = 10, max_depth = 15, min_samples_leaf = 2, n_estimators=700, random_state=42)
    rfc.fit(X, y)
#     model_selection.learning_curve(rfc, X, y)
    y_pred = rfc.predict(X_test)
    scr = rfc.score(X_test, y_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    corr = np.corrcoef(y_test.T, y_pred)
    print(scr, mae, mse, corr[0][1])
    y_pred = rfc.predict(X_valid)
    scr = rfc.score(X_valid, y_valid)
    mae = mean_absolute_error(y_valid, y_pred)
    mse = mean_squared_error(y_valid, y_pred)
    corr = np.corrcoef(y_valid.T, y_pred)
    print(scr, mae, mse, corr[0][1])
#     title = str(df['марка стали'].values[0])
#     plot_learning_curve(rfc, title, X, y, ylim=(0.7, 1.01), n_jobs=-1)
#     y_predict = rfc.predict(df_valid)
#     print('мПа:\n',y_predict*9.8)
#     print('кгс:\n',y_predict)
    tmp_df = pd.DataFrame()
    tmp_df['predict'] = list(y_pred)
    tmp_df['valid'] = y_valid.values
    score_data = {
    'MSE':[mse],
    'MAE':[mae],
    'corr':[corr]
    }

    score_df = pd.DataFrame(score_data)
    
    result_valid = valid.copy()
    result_valid['predict'] = list(y_pred)
    
    result = pd.DataFrame()
    result = pd.concat([score_df,result_valid])
    
    save_RF_model(rfc, ls_need_col, target, result, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

09Г2С
0.75584192774 1.02386123094 2.53775013266 0.905691021337
-3.2417115412 4.22147283498 21.2967596473 0.157642164201
Модель сохранена
13ХФА
0.710293132127 1.00891535572 1.77654161667 0.847610485562
0.0810396916532 1.00652111074 1.71960300744 0.63600576086
Модель сохранена
18ХМФБ
0.767632205917 1.75710920188 6.90880841348 0.898417249987
0.524487743778 2.23832233925 6.65684271534 0.926197344472
Модель сохранена
20
0.607113033881 1.17533264059 3.22072844378 0.785377737515
-0.322925615448 1.65240619667 3.57879877376 0.332932476249
Модель сохранена
30Г2
0.844340876352 1.4203064807 4.70895157183 0.921392526846
0.601025869649 2.15107113002 12.2199541256 0.83168845076
Модель сохранена
30Г2-8
0.810904130254 1.15958728237 3.11817225079 0.932614584178
-1.45621329662 4.40397216147 32.5576189578 -0.0525694217346
Модель сохранена
37Г2Ф
0.970186301487 1.14092743619 3.46156189027 0.985530167955
-3.96164440831 6.06577148689 41.8762788061 0.746739910516
Модель сохранена


In [95]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(loss='huber', learning_rate=0.1)
label = LabelEncoder() 
scaler = StandardScaler()  

for df, valid in zip(arr_df_steel, arr_valid_steel):
    print(df['марка стали'].values[0])

    y = pd.DataFrame()
    X = pd.DataFrame()
    y_valid = pd.DataFrame()
    X_valid = pd.DataFrame()
    
    target = 'Предел текучести'
#     target = 'Врем. сопротивление'

    y[target] = df[target]
    y_valid[target] = valid[target]

    for i in ls_need_col: #+ls_steel:
        X[i] = df[i]

    for i in ls_need_col: #+ls_steel:
        X_valid[i] = valid[i]
    
    X_valid = X_valid.dropna()
        
    X['марка стали'] = label.fit_transform(X['марка стали'])
    X_valid['марка стали'] = label.fit_transform(X_valid['марка стали'])
    X = scaler.fit_transform(X, y)
    X_valid = scaler.fit_transform(X_valid, y_valid)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    rfc = AdaBoostRegressor(base_estimator=gbr)
    rfc.fit(X, y)
#     model_selection.learning_curve(rfc, X, y)
    y_pred = rfc.predict(X_test)
    scr = rfc.score(X_test, y_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    corr = np.corrcoef(y_test.T, y_pred)
    print(scr, mae, mse, corr[0][1])
    y_pred = rfc.predict(X_valid)
    scr = rfc.score(X_valid, y_valid)
    mae = mean_absolute_error(y_valid, y_pred)
    mse = mean_squared_error(y_valid, y_pred)
    corr = np.corrcoef(y_valid.T, y_pred)
    print(scr, mae, mse, corr[0][1])
#     title = str(df['марка стали'].values[0])
#     plot_learning_curve(rfc, title, X, y, ylim=(0.7, 1.01), n_jobs=-1)
    y_predict = rfc.predict(df_valid)
    print('мПа:\n',y_predict*9.8)
    print('кгс:\n',y_predict)
#     tmp_df = pd.DataFrame()
#     tmp_df['predict'] = list(y_predict)
#     tmp_df['valid'] = y_valid.values
    score_data = {
    'MSE':[mse],
    'MAE':[mae],
    'corr':[corr]
    }

    score_df = pd.DataFrame(score_data)
    
#     ab = pd.concat([score_df,tmp_df])
    
    save_RF_model(rfc, ls_need_col, target, score_df, comment=' model_steel Adaboost '+str(ru2en(df['марка стали'].values[0])))

09Г2С
0.790512343625 1.38016624158 2.73484866302 0.909117092923
-2.16943776363 2.4018210099 7.56111617714 0.436128734236
мПа:
 [ 443.06500377  439.98888689  443.49781975]
кгс:
 [ 45.21071467  44.89682519  45.25487957]
Модель сохранена
13ХФА
0.497214667984 2.01995562284 5.85725993118 0.708196598528
0.0829345164094 2.04041901568 5.45158507861 0.32000995811
мПа:
 [ 418.37790931  421.85376981  417.80515087]
кгс:
 [ 42.6916234   43.04630304  42.63317866]
Модель сохранена
18ХМФБ
0.743927184011 2.6094197824 9.54180433719 0.88390654222
-0.20178158084 5.67109931135 37.6552369137 0.044512077092
мПа:
 [ 645.76694696  660.82532157  639.68532641]
кгс:
 [ 65.89458642  67.43115526  65.2740129 ]
Модель сохранена
20
0.77884616384 1.39762684199 4.96734076788 0.889975360171
0.246013067393 2.14182721482 8.1844341188 0.522660711424
мПа:
 [ 350.94653203  351.81712385  351.81712385]
кгс:
 [ 35.81087062  35.89970652  35.89970652]
Модель сохранена
30Г2
0.708611104923 2.91965417819 14.0044161555 0.842067660589


In [199]:
'''
Подбор параметров деревьев решений
'''
from sklearn.model_selection import GridSearchCV


label = LabelEncoder() 
scaler = StandardScaler()  

for df, valid in zip(arr_df_steel, arr_valid_steel):
    print(df['марка стали'].values[0])

    y = pd.DataFrame()
    X = pd.DataFrame()
    y_valid = pd.DataFrame()
    X_valid = pd.DataFrame()
    
    target = 'Предел текучести'
#     target = 'Врем. сопротивление'

    y[target] = df[target]
    y_valid[target] = valid[target]

    for i in ls_need_col: #+ls_steel:
        X[i] = df[i]

    for i in ls_need_col: #+ls_steel:
        X_valid[i] = valid[i]
    
    X_valid = X_valid.dropna()
    
        
    X['марка стали'] = label.fit_transform(X['марка стали'])
    X_valid['марка стали'] = label.fit_transform(X_valid['марка стали'])
    X = scaler.fit_transform(X, y)
    X_valid = scaler.fit_transform(X_valid, y_valid)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    tree = RandomForestRegressor()

    tree_params = {'max_depth': range(10,15),'max_features': range(12,18),  
                   'min_samples_leaf': range(1,3), 'n_estimators': [100]}

    tree_grid = GridSearchCV(tree, tree_params,cv=5,scoring='neg_mean_squared_error', verbose = True, n_jobs=-1)
    tree_grid.fit(X, y)
#     model_selection.learning_curve(tree_grid, X, y)
    y_pred = tree_grid.predict(X_test)
    scr = tree_grid.score(X_test, y_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    corr = np.corrcoef(y_test.T, y_pred)
    print(scr, mae, mse, corr[0][1])
    print(tree_grid.best_params_)
    print(tree_grid.best_score_)
    y_pred = tree_grid.predict(X_valid)
    scr = tree_grid.score(X_valid, y_valid)
    mae = mean_absolute_error(y_valid, y_pred)
    mse = mean_squared_error(y_valid, y_pred)
    corr = np.corrcoef(y_valid.T, y_pred)
    print(scr, mae, mse, corr[0][1])
#     title = str(df['марка стали'].values[0])
#     plot_learning_curve(tree_grid, title, X, y, ylim=(0.7, 1.01), n_jobs=-1)
#     y_predict = tree_grid.predict(df_valid)
#     print('мПа:\n',y_predict*9.8)
#     print('кгс:\n',y_predict)
#     tmp_df = pd.DataFrame()
#     tmp_df['predict'] = list(y_predict)
#     tmp_df['valid'] = y_valid.values
    score_data = {
    'MSE':[mse],
    'MAE':[mae],
    'corr':[corr]
    }

    score_df = pd.DataFrame(score_data)
    
#     ab = pd.concat([score_df,tmp_df])
    
    save_RF_model(tree_grid, ls_need_col, target, score_df, comment=' model_steel GS CV '+str(ru2en(df['марка стали'].values[0])))

09Г2С
Fitting 5 folds for each of 60 candidates, totalling 300 fits

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   27.0s finished



-4.18687877109 1.50513096746 4.18687877109 0.874303395926
{'max_depth': 13, 'max_features': 12, 'min_samples_leaf': 2, 'n_estimators': 100}
-12.6842390764
-5.81878235181 2.07739795686 5.81878235181 0.358095589833
Модель сохранена
13ХФА
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   40.7s finished


-4.96403336363 1.68485028384 4.96403336363 0.79352278363
{'max_depth': 10, 'max_features': 13, 'min_samples_leaf': 1, 'n_estimators': 100}
-10.010329257
-3.60917240119 1.74092357644 3.60917240119 0.660590084256
Модель сохранена
18ХМФБ
Fitting 5 folds for each of 60 candidates, totalling 300 fits


...

-8.57832046978 2.08257437156 8.57832046978 0.904879906624
{'max_depth': 14, 'max_features': 14, 'min_samples_leaf': 2, 'n_estimators': 100}
-24.6978695008
-22.8014903896 3.21778740925 22.8014903896 0.683606043065
Модель сохранена
30Г2-8
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   26.1s finished


-3.32471851744 1.2308696442 3.32471851744 0.965040612236
{'max_depth': 11, 'max_features': 15, 'min_samples_leaf': 2, 'n_estimators': 100}
-9.49943445799
-84.7311038743 7.67281816078 84.7311038743 -0.345025887165
Модель сохранена
37Г2Ф
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   29.0s finished


-9.3996626721 2.02013895391 9.3996626721 0.965334168345
{'max_depth': 14, 'max_features': 13, 'min_samples_leaf': 2, 'n_estimators': 100}
-19.4018036829
-198.464183343 13.9033991971 198.464183343 0.802591589398
Модель сохранена


In [97]:
import xgboost as xgb
from xgboost import XGBRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



'''
XGBOOST
'''
from sklearn.model_selection import GridSearchCV


label = LabelEncoder() 
scaler = StandardScaler()  

for df, valid in zip(arr_df_steel, arr_valid_steel):
    print(df['марка стали'].values[0])

    y = pd.DataFrame()
    X = pd.DataFrame()
    y_valid = pd.DataFrame()
    X_valid = pd.DataFrame()
    
    target = 'Предел текучести'
#     target = 'Врем. сопротивление'

    y[target] = df[target]
    y_valid[target] = valid[target]

    for i in ls_need_col: #+ls_steel:
        X[i] = df[i]

    for i in ls_need_col: #+ls_steel:
        X_valid[i] = valid[i]
    
    X_valid = X_valid.dropna()
        
    X['марка стали'] = label.fit_transform(X['марка стали'])
    X_valid['марка стали'] = label.fit_transform(X_valid['марка стали'])
    X = scaler.fit_transform(X, y)
    X_valid = scaler.fit_transform(X_valid, y_valid)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    xgmodel = XGBRegressor(booster='gbtree', colsample_bylevel=1,
           colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
           min_child_weight=1, missing=None, n_estimators=500,
           n_jobs=-1, nthread=None, objective='reg:linear', random_state=17,
           scale_pos_weight=1, seed=None,
           silent=True, subsample=1)

    tree_params = {'max_depth': range(10,15),'reg_alpha': range(0, 3)}
    
    params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

    tree_grid = GridSearchCV(xgmodel, params,cv=5,scoring='neg_mean_squared_error', verbose = True, n_jobs=-1)
    tree_grid.fit(X, y)
#     model_selection.learning_curve(tree_grid, X, y)
    y_pred = tree_grid.predict(X_test)
    scr = tree_grid.score(X_test, y_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    corr = np.corrcoef(y_test.T, y_pred)
    print(scr, mae, mse, corr[0][1])
    print(tree_grid.best_params_)
    print(tree_grid.best_score_)
    y_pred = tree_grid.predict(X_valid)
    scr = tree_grid.score(X_valid, y_valid)
    mae = mean_absolute_error(y_valid, y_pred)
    mse = mean_squared_error(y_valid, y_pred)
    corr = np.corrcoef(y_valid.T, y_pred)
    print(scr, mae, mse, corr[0][1])
#     title = str(df['марка стали'].values[0])
#     plot_learning_curve(tree_grid, title, X, y, ylim=(0.7, 1.01), n_jobs=-1)
#     y_predict = xgmodel.predict(df_valid)
#     print('мПа:\n',y_predict*9.8)
#     print('кгс:\n',y_predict)
#     tmp_df = pd.DataFrame()
#     tmp_df['predict'] = list(y_predict)
#     tmp_df['valid'] = y_valid.values
    score_data = {
    'MSE':[mse],
    'MAE':[mae],
    'corr':[corr]
    }

    score_df = pd.DataFrame(score_data)
    
#     ab = pd.concat([score_df,tmp_df])
    
    save_RF_model(tree_grid, ls_need_col, target, score_df, comment=' model_steel XGB  '+str(ru2en(df['марка стали'].values[0])))

09Г2С
Fitting 5 folds for each of 450 candidates, totalling 2250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 2250 out of 2250 | elapsed:  1.7min finished


-2.38396347397 1.15802729934 2.38396347397 0.903672970118
{'colsample_bytree': 0.9, 'gamma': 0.5, 'max_depth': 2, 'min_child_weight': 5, 'subsample': 0.9}
-12.2871367018
-10.0024417436 2.70471904589 10.0024417436 0.416274745779
Модель сохранена
13ХФА
Fitting 5 folds for each of 450 candidates, totalling 2250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   37.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 2250 out of 2250 | elapsed:  5.7min finished


-6.44429777786 1.93489943975 6.44429777786 0.681476838449
{'colsample_bytree': 0.6, 'gamma': 0.3, 'max_depth': 2, 'min_child_weight': 4, 'subsample': 1.0}
-10.6149560332
-7.33607465617 2.37163189565 7.33607465617 -0.132616657339
Модель сохранена
18ХМФБ
Fitting 5 folds for each of 450 candidates, totalling 2250 fits


...

-11.5121053504 2.42775639811 11.5121053504 0.872453828441
{'colsample_bytree': 1.0, 'gamma': 0.4, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.7}
-25.046107655
-23.5201163006 3.41427851725 23.5201163006 0.714910437088
Модель сохранена
30Г2-8
Fitting 5 folds for each of 450 candidates, totalling 2250 fits

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 2250 out of 2250 | elapsed:  1.2min finished



-3.75718128021 1.13470837161 3.75718128021 0.922033937921
{'colsample_bytree': 0.8, 'gamma': 0.4, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 1.0}
-13.4968905304
-84.9315748998 7.57741769155 84.9315748998 -0.265078951562
Модель сохранена
37Г2Ф
Fitting 5 folds for each of 450 candidates, totalling 2250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   58.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2250 out of 2250 | elapsed:  2.5min finished


-9.2048919396 2.29107012164 9.2048919396 0.964740092663
{'colsample_bytree': 0.7, 'gamma': 0.3, 'max_depth': 2, 'min_child_weight': 4, 'subsample': 1.0}
-23.6176465732
-222.095403566 14.7455978394 222.095403566 0.779113136313
Модель сохранена


In [22]:
def ru2en(st):
    dict_ru2en = {
        '09Г2С':'09g2s',
        '13ХФА':'13hfa',
        '18ХМФБ':'18hmfb',
        '20':'20',
        '30Г2':'30g2',
        '30Г2-8':'30g2-8',
        '30ХГМА':'30hgma',
        '30ХГМА-4':'30hgma-4',
        '37Г2Ф':'37g2f',
        '38Г2С-9':'38g2s-9'
    }
    return dict_ru2en[st]

## Обучение
    Цикл по всем отобранным маркам стали и валидации для них. Получем отдельные модели одинаковой архитектуры

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [25]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

for df, valid in zip(arr_df_steel, arr_valid_steel):
    print(df['марка стали'].values[0])

    y = pd.DataFrame()
    X = pd.DataFrame()
    y_valid = pd.DataFrame()
    X_valid = pd.DataFrame()
    
    target = 'Предел текучести'
#     target = 'Врем. сопротивление'

    y[target] = df[target]
    y_valid[target] = valid[target]

    for i in ls_need_col: #+ls_steel:
        X[i] = df[i]

    for i in ls_need_col: #+ls_steel:
        X_valid[i] = valid[i]
    
    X_valid = X_valid.dropna()
    X = scaler.fit_transform(X, y)
    X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)
    
    model = Sequential()
    model.add(Dense(units=10, input_dim=X.shape[1]))
    model.add(Activation('relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(units=4))
#     model.add(Activation('tanh'))
#     model.add(Dropout(0.3))
    
#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

    model.add(Dense(units=1))
    model.compile(
            loss=keras.losses.mean_squared_error,
            # loss=keras.losses.mean_absolute_error,
            metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
            optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.9, epsilon=1e-5, decay=1e-5)
            )
    model.fit(X, y, batch_size = 256, epochs=100, shuffle=True, verbose=0)

    y_predict = model.predict(X_valid)
    tmp_df = pd.DataFrame()
    tmp_df['predict'] = list(y_predict)
    tmp_df['valid'] = y_valid.values
    mae = mean_absolute_error(y_predict, y_valid)
    mse = mean_squared_error(y_predict, y_valid)
    corr = pearsonr(y_predict, y_valid)
    print('valid:\n',mae, mse, corr)
    y_pred = model.predict(df_valid)
    print('\non 3 str of March\nмПа:\n',y_pred*9.8)
    print('кгс:\n',y_pred)
    score_data = {
    'MSE':[mse],
    'MAE':[mae],
    'corr':[corr]
    }

    score_df = pd.DataFrame(score_data)
    
    ab = pd.concat([score_df,tmp_df])
    save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

09Г2С
valid:
 5.52753730442 43.4634899903 (array([ 0.49390909]), array([ 0.01660519]))

on 3 str of March
мПа:
 [[ 12472.85449219]
 [ 12576.50976562]
 [ 11856.25488281]]
кгс:
 [[ 1272.74023438]
 [ 1283.31726074]
 [ 1209.82189941]]
Модель сохранена
13ХФА
valid:
 3.3084074379 13.1295758 (array([-0.01917119]), array([ 0.95538458]))

on 3 str of March
мПа:
 [[ 40078.296875  ]
 [ 38544.52734375]
 [ 37110.390625  ]]
кгс:
 [[ 4089.62207031]
 [ 3933.11499023]
 [ 3786.7746582 ]]
Модель сохранена
18ХМФБ
valid:
 13.6471835003 226.398038877 (array([ 0.74760465]), array([ 0.00023362]))

on 3 str of March
мПа:
 [[-21096.46679688]
 [-19927.58203125]
 [-19836.54492188]]
кгс:
 [[-2152.70068359]
 [-2033.42675781]
 [-2024.13720703]]
Модель сохранена
20
valid:
 21.7644965762 483.248735472 (array([ 0.49389807]), array([ 0.02287123]))

on 3 str of March
мПа:
 [[ 65883.5625    ]
 [ 65954.4296875 ]
 [ 62340.13671875]]
кгс:
 [[ 6722.81201172]
 [ 6730.04345703]
 [ 6361.23828125]]
Модель сохранена
30Г2
valid:
 7

# Отдельные модели на каждую марку стали

## 09Г2С

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [55]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[0]
valid = arr_valid_steel[0]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
#     target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)

model = Sequential()
model.add(Dense(units=4, input_dim=X.shape[1]))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(units=4))
# model.add(Activation('tanh'))
# model.add(Dropout(0.1))

model.add(Dense(units=1))
model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
        optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.8, epsilon=1e-1, decay=1e-7)
        )
model.fit(X, y, batch_size = 64, epochs=50, shuffle=True)

y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])
save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

Epoch 1/50
420/420 [==============================] - 1s 2ms/step - loss: 947.7293 - mean_absolute_error: 26.9945
Epoch 2/50
420/420 [==============================] - 0s 48us/step - loss: 1157.8369 - mean_absolute_error: 26.2844
Epoch 3/50
420/420 [==============================] - 0s 45us/step - loss: 450.5376 - mean_absolute_error: 16.4009
Epoch 4/50
420/420 [==============================] - 0s 60us/step - loss: 195.1016 - mean_absolute_error: 10.6912
Epoch 5/50
420/420 [==============================] - 0s 72us/step - loss: 225.7731 - mean_absolute_error: 12.4758
Epoch 6/50
420/420 [==============================] - 0s 62us/step - loss: 266.1887 - mean_absolute_error: 14.1134
Epoch 7/50
420/420 [==============================] - 0s 62us/step - loss: 256.7849 - mean_absolute_error: 12.8597
Epoch 8/50
420/420 [==============================] - 0s 62us/step - loss: 147.1587 - mean_absolute_error: 9.9875
Epoch 9/50
420/420 [==============================] - 0s 76us/step - loss: 348.46

In [276]:
%matplotlib inline
#увеличим дефолтный размер графиков
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 10
rcParams.update({'font.size': 20})

import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
# updatable plot
# a minimal example (sort of)

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="Ошибка")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

# class PlotLearning(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.i = 0
#         self.x = []
#         self.losses = []
#         self.val_losses = []
#         self.acc = []
#         self.val_acc = []
#         self.fig = plt.figure()
        
#         self.logs = []

#     def on_epoch_end(self, epoch, logs={}):
        
#         self.logs.append(logs)
#         self.x.append(self.i)
#         self.losses.append(logs.get('loss'))
#         self.val_losses.append(logs.get('val_loss'))
#         self.acc.append(logs.get('acc'))
#         self.val_acc.append(logs.get('val_acc'))
#         self.i += 1
#         f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
#         clear_output(wait=True)
        
#         ax1.set_yscale('log')
#         ax1.plot(self.x, self.losses, label="loss")
#         ax1.plot(self.x, self.val_losses, label="val_loss")
#         ax1.legend()
        
#         ax2.plot(self.x, self.acc, label="accuracy")
#         ax2.plot(self.x, self.val_acc, label="validation accuracy")
#         ax2.legend()
        
#         plt.show();
        
# plot_losses = PlotLearning()

## 13ХФА

In [251]:
now_code = datetime.now()
time_code ='MAE1.9_MSE6_13hfa_'+ "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

'MODELS_Code/MAE1.9_MSE6_13hfa_5_4date14_6_7_8time.py'

In [103]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[1]
valid = arr_valid_steel[1]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
#     target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = Sequential()
model.add(Dense(units=16, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(units=1))
# model.add(Activation('tanh'))
# model.add(Dropout(0.3))

#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

model.add(Dense(units=1))

model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
        optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.8, epsilon=1e-2, decay=1e-7)
        )
model.fit(X_train, y_train, batch_size = 64, epochs=100, shuffle=True)

y_predict = model.predict(X_test)
mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
corr = pearsonr(y_predict, y_test)
print('test:\n',mae, mse, corr)
y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])


Epoch 1/100
1787/1787 [==============================] - 2s 1ms/step - loss: 398.5963 - mean_absolute_error: 15.8833
Epoch 2/100
1787/1787 [==============================] - 0s 58us/step - loss: 146.4686 - mean_absolute_error: 9.8598
Epoch 3/100
1787/1787 [==============================] - 0s 58us/step - loss: 129.8395 - mean_absolute_error: 9.4065
Epoch 4/100
1787/1787 [==============================] - 0s 56us/step - loss: 125.9098 - mean_absolute_error: 9.2240
Epoch 5/100
1787/1787 [==============================] - 0s 57us/step - loss: 103.4846 - mean_absolute_error: 8.4697
Epoch 6/100
1787/1787 [==============================] - 0s 59us/step - loss: 103.5072 - mean_absolute_error: 8.6700
Epoch 7/100
1787/1787 [==============================] - 0s 57us/step - loss: 80.4697 - mean_absolute_error: 7.2994
Epoch 8/100
1787/1787 [==============================] - 0s 57us/step - loss: 73.8880 - mean_absolute_error: 7.0170
Epoch 9/100
1787/1787 [==============================] - 0s 59us/s

In [104]:
save_NN_model(model, ls_need_col, target, ab, comment='BEST valid MAE2 MSE6 model_steel '+str(ru2en(df['марка стали'].values[0])))

Модель сохранена


## 18ХМФБ

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [82]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[2]
valid = arr_valid_steel[2]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
#     target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)

model = Sequential()
model.add(Dense(units=4, input_dim=X.shape[1]))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(units=4))
# model.add(Activation('tanh'))
# model.add(Dropout(0.3))

#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

model.add(Dense(units=1))
model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
        optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.8, epsilon=1e-3, decay=1e-3)
        )
model.fit(X, y, batch_size = 64, epochs=100, shuffle=True)

y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])
save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

Epoch 1/100
2229/2229 [==============================] - 2s 1ms/step - loss: 1529.0278 - mean_absolute_error: 29.6527
Epoch 2/100
2229/2229 [==============================] - 0s 63us/step - loss: 173.2917 - mean_absolute_error: 9.1665
Epoch 3/100
2229/2229 [==============================] - 0s 65us/step - loss: 73.8105 - mean_absolute_error: 5.8532
Epoch 4/100
2229/2229 [==============================] - 0s 67us/step - loss: 50.9426 - mean_absolute_error: 5.0326
Epoch 5/100
2229/2229 [==============================] - 0s 63us/step - loss: 43.8407 - mean_absolute_error: 4.7203
Epoch 6/100
2229/2229 [==============================] - 0s 67us/step - loss: 40.6478 - mean_absolute_error: 4.4986
Epoch 7/100
2229/2229 [==============================] - 0s 68us/step - loss: 36.1614 - mean_absolute_error: 4.3548
Epoch 8/100
2229/2229 [==============================] - 0s 66us/step - loss: 34.7695 - mean_absolute_error: 4.3150
Epoch 9/100
2229/2229 [==============================] - 0s 67us/step

## 20

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [ ]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[3]
valid = arr_valid_steel[3]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
#     target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)

model = Sequential()
model.add(Dense(units=32, input_dim=X.shape[1]))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(units=4))
model.add(Activation('tanh'))
model.add(Dropout(0.3))

#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

model.add(Dense(units=1))
model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
        optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.9, epsilon=1e-5, decay=1e-5)
        )
model.fit(X, y, batch_size = 256, epochs=100, shuffle=True, verbose=0)

y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])
save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

## 30Г2

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [297]:
arr_df_steel[4] = pd.read_excel('30g2_prepared.xlsx')
arr_valid_steel[4] = pd.read_excel('30g2_valid_prepared.xlsx')

In [298]:
arr_df_steel[4].shape, arr_valid_steel[4].shape

((10965, 26), (223, 26))

In [121]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[4]
valid = arr_valid_steel[4]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
# target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = Sequential()
model.add(Dense(units=4, input_dim=X_train.shape[1]))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(units=1))
model.add(Activation('tanh'))
# model.add(Dropout(0.5))

#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

model.add(Dense(units=1))
model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
            optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
#         optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.6, epsilon=1e-3, decay=1e-3)
        )
model.fit(X_train, y_train, batch_size = 16, epochs=100, shuffle=True)

y_predict = model.predict(X_test)
mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
corr = pearsonr(y_predict, y_test)
print('test:\n',mae, mse, corr)

y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])


Epoch 1/100
2833/2833 [==============================] - 2s 862us/step - loss: 126.1868 - mean_absolute_error: 6.9447
Epoch 2/100
2833/2833 [==============================] - 1s 250us/step - loss: 45.5547 - mean_absolute_error: 5.5266
Epoch 3/100
2833/2833 [==============================] - 1s 250us/step - loss: 45.1698 - mean_absolute_error: 5.4987
Epoch 4/100
2833/2833 [==============================] - ETA: 0s - loss: 46.1607 - mean_absolute_error: 5.53 - 1s 251us/step - loss: 45.6878 - mean_absolute_error: 5.5117
Epoch 5/100
2833/2833 [==============================] - 1s 252us/step - loss: 45.2283 - mean_absolute_error: 5.4891
Epoch 6/100
2833/2833 [==============================] - 1s 250us/step - loss: 45.6304 - mean_absolute_error: 5.5120
Epoch 7/100
2833/2833 [==============================] - 1s 245us/step - loss: 45.5253 - mean_absolute_error: 5.4987
Epoch 8/100
2833/2833 [==============================] - 1s 250us/step - loss: 45.6633 - mean_absolute_error: 5.5110
Epoch 9/1

In [ ]:
save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

In [390]:
valid = arr_valid_steel[3]
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

# target = 'Предел текучести'
target = 'Врем. сопротивление'

y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()

X_valid = scaler.fit_transform(X_valid, y_valid)
y_predict = model.predict(X_valid)
mae = (mean_absolute_error(y_predict, y_valid))
mse = (mean_squared_error(y_predict,  y_valid))
corr = (pearsonr(y_predict,  y_valid))
print(np.mean(mae), np.mean(mse), np.mean(corr))

23.566473098 566.999710165 -0.268100486836


In [383]:
save_NN_model(model, ls_need_col, target, ab, comment='test mae1.8 mse7.3 model_steel '+str(ru2en(df['марка стали'].values[0])))

Модель сохранена


In [113]:
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[4]
valid = arr_valid_steel[4]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
# target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)

rfc = RandomForestRegressor(criterion='mse',max_features = 15, max_depth = 15, min_samples_leaf = 2, n_estimators=500, random_state=42, 
                            n_jobs=-1, verbose=1, oob_score=True)
rfc.fit(X, y)
model_selection.learning_curve(rfc, X, y)
y_pred = rfc.predict(X_valid)
scr = rfc.score(X_valid, y_valid)
mae = mean_absolute_error(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)
corr = np.corrcoef(y_valid.T, y_pred)
print(scr, mae, mse, corr[0][1])
title = str(df['марка стали'].values[0])
y_predict = rfc.predict(df_valid)
print('мПа:\n',y_predict*9.8)
print('кгс:\n',y_predict)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)

ab = pd.concat([score_df,tmp_df])

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.6s finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapse

[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | e

0.440500687679 3.20378752149 22.5912375363 0.69865125064
мПа:
 [ 619.05533868  615.50450354  615.50450354]
кгс:
 [ 63.16891211  62.80658199  62.80658199]


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished


In [ ]:
plot_learning_curve(rfc, title, X, y, ylim=(0.7, 1.01), n_jobs=-1)

In [353]:
save_RF_model(rfc, ls_need_col, target, ab, comment=' MSE12 MAE2.5 model_steel '+str(ru2en(df['марка стали'].values[0])))

Модель сохранена


## 30Г2-8

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [271]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[6]
valid = arr_valid_steel[6]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
#     target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)

model = Sequential()
model.add(Dense(units=8, input_dim=X.shape[1]))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(units=2))
# model.add(Activation('tanh'))
# model.add(Dropout(0.3))

#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

model.add(Dense(units=1))
model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
        optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.9, epsilon=1e-3, decay=1e-3)
        )
model.fit(X, y, batch_size = 256, epochs=100, shuffle=True)

y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])
save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

Epoch 1/100
98/98 [==============================] - 12s 120ms/step - loss: 5701.7354 - mean_absolute_error: 74.6518
Epoch 2/100
98/98 [==============================] - 0s 51us/step - loss: 5497.0308 - mean_absolute_error: 73.4086
Epoch 3/100
98/98 [==============================] - 0s 72us/step - loss: 5351.1079 - mean_absolute_error: 72.5217
Epoch 4/100
98/98 [==============================] - 0s 113us/step - loss: 5118.4683 - mean_absolute_error: 71.0712
Epoch 5/100
98/98 [==============================] - 0s 123us/step - loss: 4768.8198 - mean_absolute_error: 68.7179
Epoch 6/100
98/98 [==============================] - 0s 112us/step - loss: 4292.7598 - mean_absolute_error: 65.1476
Epoch 7/100
98/98 [==============================] - 0s 123us/step - loss: 3715.0571 - mean_absolute_error: 60.0421
Epoch 8/100
98/98 [==============================] - 0s 113us/step - loss: 3106.2632 - mean_absolute_error: 53.7473
Epoch 9/100
98/98 [==============================] - 0s 113us/step - loss

Epoch 73/100
98/98 [==============================] - 0s 153us/step - loss: 48.0646 - mean_absolute_error: 5.2982
Epoch 74/100
98/98 [==============================] - 0s 143us/step - loss: 47.2981 - mean_absolute_error: 5.4979
Epoch 75/100
98/98 [==============================] - 0s 154us/step - loss: 47.2903 - mean_absolute_error: 5.2535
Epoch 76/100
98/98 [==============================] - 0s 153us/step - loss: 47.1275 - mean_absolute_error: 5.4945
Epoch 77/100
98/98 [==============================] - 0s 184us/step - loss: 46.9302 - mean_absolute_error: 5.2354
Epoch 78/100
98/98 [==============================] - 0s 174us/step - loss: 47.3645 - mean_absolute_error: 5.5196
Epoch 79/100
98/98 [==============================] - 0s 153us/step - loss: 46.9033 - mean_absolute_error: 5.2299
Epoch 80/100
98/98 [==============================] - 0s 153us/step - loss: 46.5953 - mean_absolute_error: 5.4450
Epoch 81/100
98/98 [==============================] - 0s 144us/step - loss: 45.7948 - me

## 30ХГМА

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [ ]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[0]
valid = arr_valid_steel[0]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
#     target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)

model = Sequential()
model.add(Dense(units=32, input_dim=X.shape[1]))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(units=4))
model.add(Activation('tanh'))
model.add(Dropout(0.3))

#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

model.add(Dense(units=1))
model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
        optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.9, epsilon=1e-5, decay=1e-5)
        )
model.fit(X, y, batch_size = 256, epochs=100, shuffle=True, verbose=0)

y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])
save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

## 30ХГМА-4

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [ ]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[0]
valid = arr_valid_steel[0]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
#     target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)

model = Sequential()
model.add(Dense(units=32, input_dim=X.shape[1]))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(units=4))
model.add(Activation('tanh'))
model.add(Dropout(0.3))

#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

model.add(Dense(units=1))
model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
        optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.9, epsilon=1e-5, decay=1e-5)
        )
model.fit(X, y, batch_size = 256, epochs=100, shuffle=True, verbose=0)

y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])
save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))

## 37Г2Ф

In [ ]:
now_code = datetime.now()
time_code = "%d_%ddate%d_%d_%d_%dtime.py" % (now_code.day, now_code.month, now_code.hour, now_code.minute, 
                                          now_code.second, now_code.microsecond/100000)
modelfilename = 'MODELS_Code/'+time_code
modelfilename

In [ ]:
# %%writefile $modelfilename
label = LabelEncoder() 
scaler = StandardScaler()  

df = arr_df_steel[0]
valid = arr_valid_steel[0]

y = pd.DataFrame()
X = pd.DataFrame()
y_valid = pd.DataFrame()
X_valid = pd.DataFrame()

target = 'Предел текучести'
#     target = 'Врем. сопротивление'

y[target] = df[target]
y_valid[target] = valid[target]

for i in ls_need_col: #+ls_steel:
    X[i] = df[i]

for i in ls_need_col: #+ls_steel:
    X_valid[i] = valid[i]

X_valid = X_valid.dropna()
X = scaler.fit_transform(X, y)
X_valid = scaler.fit_transform(X_valid, y_valid)
#     X_train, X_valid, y_train, y_valid = train_valid_split(X, y, valid_size=0.2)

model = Sequential()
model.add(Dense(units=32, input_dim=X.shape[1]))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(units=4))
model.add(Activation('tanh'))
model.add(Dropout(0.3))

#     model.add(Dense(units=16, input_dim=X.shape[1]))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.6))
#     model.add(Dense(units=2))
#     model.add(Activation('tanh'))

model.add(Dense(units=1))
model.compile(
        loss=keras.losses.mean_squared_error,
        # loss=keras.losses.mean_absolute_error,
        metrics=[keras.metrics.mean_absolute_error],
#             optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.6, decay=1e-7, nesterov=True)
#             optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.75, beta_2=0.999, epsilon=1e-5, decay=1e-7, amsgrad=True)
        optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.9, epsilon=1e-5, decay=1e-5)
        )
model.fit(X, y, batch_size = 256, epochs=100, shuffle=True, verbose=0)

y_predict = model.predict(X_valid)
tmp_df = pd.DataFrame()
tmp_df['predict'] = list(y_predict)
tmp_df['valid'] = y_valid.values
mae = mean_absolute_error(y_predict, y_valid)
mse = mean_squared_error(y_predict, y_valid)
corr = pearsonr(y_predict, y_valid)
print('valid:\n',mae, mse, corr)
y_pred = model.predict(df_valid)
print('\non 3 str of March\nмПа:\n',y_pred*9.8)
print('кгс:\n',y_pred)
score_data = {
'MSE':[mse],
'MAE':[mae],
'corr':[corr]
}

score_df = pd.DataFrame(score_data)
    
ab = pd.concat([score_df,tmp_df])
save_NN_model(model, ls_need_col, target, ab, comment=' model_steel '+str(ru2en(df['марка стали'].values[0])))